In [69]:
import pandas as pd
import numpy as np
import math
import datetime
import subprocess
import bandit.main as bd

In [70]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [71]:
# todo: combine bandit_class and bandit_diff
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [72]:
def df_bandit_diff(df):
    dfgb = df.groupby(['goal'])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [73]:
subprocess.Popen('GoalSetter.xlsx', shell=True)

In [74]:
df = get_frame('GoalSetter.xlsx')

In [75]:
df = bd.reduce(df,'goal')

21 0.9475 20


In [76]:
df_quality = bd.get_quality(df['score'])

In [77]:
df = df.join(df_quality).sort_values('Quality', ascending=False)

In [78]:
#df.loc[df.score.isnull(), 'Quality'] = 0.5

In [79]:
df.sort_values('date', ascending=True).tail(10)

,date,goal,cohort,complete,score,Quality
31,2018-09-06,10,1,1.0,0.000000,0.000000
32,2018-09-07,9,1,21.0,13.966610,0.541728
33,2018-09-08,7,1,13.0,10.330412,0.400690
34,2018-09-09,7,1,11.0,10.330412,0.400690
35,2018-09-10,9,1,2.0,0.000000,0.000000
36,2018-09-11,4,0,1.0,0.000000,0.000000
37,2018-09-12,19,2,10.0,0.000000,0.000000
38,2018-09-13,13,1,0.0,0.000000,0.000000
39,2018-09-14,8,1,8.0,12.125733,0.470325
40,2018-09-15,11,1,4.0,0.000000,0.000000


In [80]:
df = df.sort_values('Quality', ascending=False)
df.head(10)

,date,goal,cohort,complete,score,Quality
14,2018-08-19,15,2,15.0,25.781579,1.000000
21,2018-08-26,13,1,16.0,21.713609,0.842214
20,2018-08-25,13,1,15.0,21.713609,0.842214
12,2018-08-24,13,1,14.0,21.713609,0.842214
24,2018-08-30,13,1,13.0,21.713609,0.842214
22,2018-08-27,10,1,11.0,15.848932,0.614739
27,2018-09-02,10,1,10.0,15.848932,0.614739
9,2018-08-15,10,1,14.0,15.848932,0.614739
29,2018-09-04,10,1,10.0,15.848932,0.614739
8,2018-08-14,9,1,10.0,13.966610,0.541728


In [81]:
df_class = df_bandit_class(df,'cohort')

In [82]:
df_class

,cohort,count,mean,var,total,donext
1,1,26,0.387223,0.099509,36,0.572849
2,2,6,0.166667,0.166667,36,0.553077
0,0,4,0.068213,0.003948,36,0.541468


In [83]:
nextClass = df_class.iloc[0,0]

In [84]:
if df['complete'].isnull().sum() > 0:
    df_filtered = df[df['cohort'] == nextClass]
else:
    df_filtered = df

In [85]:
df_bandit_diff(df_filtered)

,goal,count,mean,var,total,donext
5,12,1,0.000000,NaN,26,0.902510
3,10,5,0.491791,0.075581,26,0.895406
1,8,2,0.235163,0.110603,26,0.873334
0,7,4,0.400690,0.000000,26,0.851945
2,9,5,0.433383,0.058694,26,0.836997
6,13,7,0.481265,0.202664,26,0.822382
4,11,2,0.000000,0.000000,26,0.638171
